In [1]:
import pandas as pd

df = pd.read_csv('final_data.csv')

shakespeare_txt = ''
with open('shakespeare.txt') as f:
    for line in f:
        shakespeare_txt += line
        

In [2]:
import re
import unicodedata
import contractions
from nltk import word_tokenize
import numpy as np

'''
w3_lst = []
w1_w2_lst = []
trigram_mat = []
'''

def cleaner(text):
    text = re.sub(r'([A-Z]+[a-z]+\s)?[A-Z]+[a-z]+:','', text)
    text = re.sub('-+',' ',text)
    text = unicodedata.normalize('NFKD',text).encode('ascii','ignore').decode('utf-8')
    text = contractions.fix(text)
    text = text.lower()
    text = re.sub(r'[\r|\n|\r\n]+', '\n', text)
    text = re.sub(r'[^a-zA-Z\s\.\!\?]', ' ', text)
    
    
    text = re.sub(' +', ' ', text)
    
    tokens = word_tokenize(text)
    
    return tokens

def get_ngrams(text,n):
    ngrams = []
    for i in range(len(text)-n):
        ngrams.append(text[i:i+n])
        
    return ngrams


In [3]:
cleaned_text = [cleaner(i) for i in df.text]
shakespeare = cleaner(shakespeare_txt)
cleaned_text = [item for items in cleaned_text for item in items]
cleaned_text = cleaned_text + shakespeare


In [4]:
trigrams = get_ngrams(cleaned_text,3)
bigrams = get_ngrams(cleaned_text,2)

In [5]:
trigram_cnt = {}
bigram_cnt = {}

for trigram in trigrams:
    trigram = (' ').join(trigram)
    if trigram in trigram_cnt:
        trigram_cnt[trigram] += 1
    else:
        trigram_cnt[trigram] = 1
        
for bigram in bigrams:
    bigram = (' ').join(bigram)
    if bigram in bigram_cnt:
        bigram_cnt[bigram] += 1
    else:
        bigram_cnt[bigram] = 1

In [6]:
vocab = len(set(cleaned_text))
probabilities = {}

for trigram, cnt in trigram_cnt.items():
    bigram = trigram.split(' ')
    bigram = bigram[0] + ' ' + bigram[1]
    prob = (cnt + 1) / (bigram_cnt[bigram] + vocab)
    if trigram not in probabilities:
        probabilities[trigram] = prob
    else:
        print('oops! somethings wrong')

In [10]:
import random

def get_random_token(matches):
    tokens = []
    probs = []
    
    for k,v in matches.items():
        tokens.append(k)
        probs.append(v)
    
    if len(tokens) > 0:
        token = random.choices(tokens,weights=probs,k=1)
    else: 
        token = random.choices(cleaned_text,k=1)
    
    return token[0].split(' ')[-1]
    

def generate_text(text, n):
    output = []
    clean_text = cleaner(text)
    output = [x for x in clean_text]
    clean_text = clean_text[-2:]
    clean_text = (' ').join(clean_text)
    
    while(n != 0):
        matches = {}

        for k,v in probabilities.items():
            bigram = (' ').join(k.split(' ')[:-1])
            if clean_text == bigram:
                matches[k] = v

        token = get_random_token(matches)
        output.append(token)
        if (token == '.'): break;
        clean_text = clean_text.split(' ')[-1:]
        clean_text.append(token)
        clean_text = (' ').join(clean_text)
        
        n -= 1
    
    print((' ').join(output))

In [16]:
text = input('Please enter text\n')
#n = int(input('Please enter length of text (number of words to generate)'))
print()
generate_text(text, 100)

Please enter text
To be

to be found which will carry no crotchets i will confine myself to give your grace humbly on my fellows he says the king i was anointed king is most absurdly null as regards sequence of because and most devoted of women s rheum which are continually astonishing the skeptic .
